In [1]:
import os
import re
import pandas as pd

In [2]:
def calculate_life_expectancy(age):
    # Calculate life expectancy
    if age < 0.08:
        return 89.99
    elif 0.08 <= age < 1:
        return 89.55
    elif 1 <= age < 5:
        return 89.07
    elif 5 <= age < 10:
        return 82.58
    elif 10 <= age < 15:
        return 77.58
    elif 15 <= age < 20:
        return 72.60
    elif 20 <= age < 25:
        return 67.62
    elif 25 <= age < 30:
        return 62.66
    elif 30 <= age < 35:
        return 57.71
    elif 35 <= age < 40:
        return 52.76
    elif 40 <= age < 45:
        return 47.83
    elif 45 <= age < 50:
        return 42.94
    elif 50 <= age < 55:
        return 38.10
    elif 55 <= age < 60:
        return 33.33
    elif 60 <= age < 65:
        return 28.66
    elif 65 <= age < 70:
        return 24.12
    elif 70 <= age < 75:
        return 19.76
    elif 75 <= age < 80:
        return 15.65
    elif 80 <= age < 85:
        return 11.69
    else:
        return 7.05

In [3]:
data_folder_raw = "data/raw"

# Select the folder where the raw files are located
current_directory = "C:/projetos/machine_learning_time_series_yll/"
file_path = os.path.join(current_directory, data_folder_raw)
# List of ICD-10 codes belonging to ICSAPS
cod_icsaps = ["A37","A36","A33","A34","A35","B26","B06","B05","A95","B16","G000","A170","A19","A150","A151","A152","A153","A160",
                "A161","A162","A154","A155","A156","A157","A158","A159","A163","A164","A165","A166","A167","A168","A169","A171","A172",
                "A173","A174","A175","A176","A177","A178","A179","A18","I00","I01","I02","A51","A52","A53","B50","B51","B52","B53",
                "B54","B77","E86","A00","A01","A02","A03","A04","A05","A06","A07","A08","A09","D50","E40","E41","E42","E43","E44","E45",
                "E46","E50","E51","E52","E53","E54","E55","E56","E57","E58","E59","E60","E61","E62","E63","E64","H66","J00","J01","J02",
                "J03","J06","J31","J13","J14","J153","J154","J158","J159","J181","J45","J46","J20","J21","J40","J41","J42","J43","J47",
                "J44","I10","I11","I20","I50","J81","I63","I64","I65","I66","I67","I69","G45","G46","E100","E101","E110","E111","E120",
                "E121","E130","E131","E140","E141","E102","E103","E104","E105","E106","E107","E108","E112","E113","E114","E115","E116",
                "E117","E118","E122","E123","E124","E125","E126","E127","E128","E132","E133","E134","E135","E136","E137","E138","E142",
                "E143","E144","E145","E146","E147","E148","E109","E119","E129","E139","E149","G40","G41","N10","N11","N12","N30","N34",
                "N390","A46","L01","L02","L03","L04","L08","N70","N71","N72","N73","N75","N76","K25","K26","K27","K28","K920","K921",
                "K922","O23","A50","P350"]
# List with the dataframes already processed
dfs = []

print("--------------------------------------------------------------------------")
print("Carregando os dados dos arquivos extraídos, tratando e concatenando...")
# Generate the dataframe
pattern = r'^Mortalidade_Geral_\d{4}\.csv$'
for file in os.listdir(file_path):
    if re.match(pattern, file):
        # Read CSV file with Pandas
        df = pd.read_csv(os.path.join(file_path, file), delimiter=';', encoding='ISO-8859-1', low_memory=False)
        # Analyze whether the ICD-10 code belongs to ICSAPs
        df['icsaps'] = df['CAUSABAS'].apply(lambda x: 'Sim' if x in cod_icsaps else 'Não')
        # Keep only data that is ICSAPs
        df = df[df['icsaps'] == 'Sim']
        # Perform transformation of birth and death dates
        df['dt_obito'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')
        df['dt_nasc'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y', errors='coerce')
        # Delete null data for date of birth and date of death
        df = df.dropna(subset=['dt_nasc'])
        df = df.dropna(subset=['dt_obito'])
        # Create the age column
        df['idade'] = ((df['dt_obito'] - df['dt_nasc']).dt.days / 365.25).round(2)
        # Keep only data with valid ages
        df = df[df['idade'] >= 0]
        # Create column yll
        df['yll'] = df.apply(lambda row: calculate_life_expectancy(row['idade']), axis=1)
        # Create the columns ano_obito and quad_obito
        df['ano_obito'] = df['dt_obito'].dt.year.astype(float).astype(pd.Int64Dtype()).astype(str).where(df['dt_obito'].notna())
        df['quad_obito'] = pd.cut(df['dt_obito'].dt.month, bins=[1, 5, 9, 13], labels=[1, 2, 3], right=False)
        df['quad_obito'] = df['quad_obito'].astype('object')
        # Extract the first 6 digits from the CODMUNRES column
        df['cd_mun_res'] = df['CODMUNRES'].astype(str).str.slice(stop=6)
        # Rename columns
        df = df.rename(columns={'CAUSABAS':'cid10'})
        # Select desired columns
        df = df[['ano_obito','quad_obito','dt_obito','dt_nasc','idade','yll','cid10','cd_mun_res']]
        # Add the dataframe to the list of dataframes
        dfs.append(df)

# Concatenate the dataframes into a single final dataframe
df_group = pd.concat(dfs, ignore_index=True)

--------------------------------------------------------------------------
Carregando os dados dos arquivos extraídos, tratando e concatenando...


In [4]:
df_group.head()

,ano_obito,quad_obito,dt_obito,dt_nasc,idade,yll,cid10,cd_mun_res
0,2010,2,2010-05-30,1937-09-04,72.73,19.76,I64,120070
1,2010,2,2010-06-08,1916-12-04,93.51,7.05,I10,120040
2,2010,1,2010-02-20,1975-04-28,34.82,57.71,J81,120045
3,2010,1,2010-03-11,1915-10-12,94.41,7.05,E149,120040
4,2010,2,2010-05-14,1918-09-20,91.65,7.05,I10,120040


In [5]:
df_group.dtypes

ano_obito             object
quad_obito            object
dt_obito      datetime64[ns]
dt_nasc       datetime64[ns]
idade                float64
yll                  float64
cid10                 object
cd_mun_res            object
dtype: object

In [20]:
def porte(populacao):
    # Definir o porte pelo tamanho da população
    if populacao <= 20000:
        return 'Pequeno Porte I'
    elif populacao >= 20001 and populacao <= 50000:
        return 'Pequeno Porte II'
    elif populacao >= 50001 and populacao <= 100000:
        return 'Médio Porte'
    elif populacao >= 100001 and populacao <= 900000:
        return 'Grande Porte'
    elif populacao >= 900001:
        return 'Metrópole'

In [23]:
data_folder_raw = "data/raw"
current_directory = "C:/projetos/machine_learning_time_series_yll/"
file_path = os.path.join(current_directory, data_folder_raw)

pattern = r'.*ibge_cnv.*'
for file in os.listdir(file_path):
    if re.match(pattern, file):
        # Create population dataframe
        df = pd.read_csv(os.path.join(file_path, file), skiprows=3, sep=';', encoding='ISO-8859-1', low_memory=False)
        index_total = df[df['Município'] == 'Total'].index[0]
        df = df.loc[:index_total-1]
        # Creating the cod_municipio column with the first 6 characters of the Municipio column
        df['cd_municipio'] = df['Município'].str[:6]
        # Transforming column years into rows with corresponding population values
        df = pd.melt(df, id_vars=['cd_municipio'], value_vars=[str(year) for year in range(2010, 2020)],var_name='ano', value_name='populacao')
        # Convert 'populacao' column to integer
        df['populacao'] = pd.to_numeric(df['populacao'], errors='coerce').fillna(0).astype(int)
        # Applying the 'porte' function
        df['porte'] = df['populacao'].apply(porte)

In [24]:
df.head()

,cd_municipio,ano,populacao,porte
0,110001,2010,24422,Pequeno Porte II
1,110002,2010,90354,Médio Porte
2,110003,2010,6309,Pequeno Porte I
3,110004,2010,78601,Médio Porte
4,110005,2010,17030,Pequeno Porte I


In [8]:
import os
import re
import pandas as pd

data_folder_raw = "data\\raw"
# current_directory = "C:/projetos/machine_learning_time_series_yll/"
file_path = os.path.join(os.getcwd(), data_folder_raw)

pattern = "municipios.csv"
for file in os.listdir(file_path):
    if re.match(pattern, file):
        # Create population dataframe
        df = pd.read_csv(os.path.join(file_path, file), sep=';', encoding='ISO-8859-1', low_memory=False)
        # Rename columns
        df = df.rename(columns={
            'CÓDIGO DO MUNICÍPIO - IBGE':'cd_municipio',
            'MUNICÍPIO - IBGE':'nm_municipio',
            'UF':'sl_uf'
            })
        # Convert to string if necessary
        df['cd_municipio'] = df['cd_municipio'].astype(str)
        df['cd_municipio'] = df['cd_municipio'].str[:6]
        df['cd_uf'] = df['cd_municipio'].str[:2]
        # Select desired columns
        df = df[['cd_uf','sl_uf','cd_municipio','nm_municipio']]

df.head()

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'c:\\projetos\\machine_learning_time_series_yll\\tests\\data\\raw'

In [5]:
os.getcwd()

'c:\\projetos\\machine_learning_time_series_yll\\tests'

In [15]:
df.dtypes

cd_municipio     int64
nm_municipio    object
sl_uf           object
dtype: object

In [ ]:
# exibe o dataframe final
df_group.head()

In [ ]:
df_group.dtypes

In [ ]:
df_group.columns

In [ ]:
df_group.info()

In [ ]:
df_group.shape

In [ ]:
df_group.isna().sum()